We solve the advection dominated skew to mesh benchmark problem as described in [links go here] using the various methods encountered in class:

$$-\kappa \Delta u + \beta\cdot \nabla u = 0  $$

with $\kappa = 10^{-3}$, $\beta = (.5, 1)$ on a square grid with dirichlet boundary condition 

$$u = \begin{cases} 1 & y = 0 \text{ or } x = 0 \text{ and } y < 0.2 \\ 0 & \text{otherwise} \end{cases}   $$ 

In [3]:
import nutils as ntl
import numpy as np
import matplotlib.pyplot as plt
import inspect


## SUPG

We use SUPG to solve the advection dominated skew to mesh benchmark problem as described in [links go here]:

$$-\kappa \Delta u + \beta\cdot \nabla u = 0  $$

with $\kappa = 10^{-3}$, $\beta = (.5, 1)$ on a square grid with dirichlet boundary condition 

$$u = \begin{cases} 1 & y = 0 \text{ or } x = 0 \text{ and } y < 0.2 \\ 0 & \text{otherwise} \end{cases}   $$ 

In [66]:
degree = 1 #degree of basis functions
nelems = 20 #number of elements along edge of unit square
etype = 'square'

topo, geom =  ntl.mesh.unitsquare(nelems, etype) #rectilinear([np.linspace(0,1, nx), np.linspace(0,1, ny) ])
ns = ntl.function.Namespace()
ns.x = geom
ns.beta= np.array([0.5, 1.0])
ns.kappa = 1e-3
ns.basis = topo.basis('lagrange', degree = degree)
ns.u = 'basis_i ?dofs_i'

#element intrinsic time scale
ns.h = 0.5*np.sqrt(2/nelems**2) #mesh size parameter
ns.betanorm = 'beta_i beta_i'
ns.tau = 'h / (2 betanorm)'

#dirichlet boundary condition
ns.badbnd = topo.boundary['left'].indicator(topo.boundary['left'][0:int(nelems/2)])

'''
print(topo.boundary['left'][0:int(nelems/5)])    
bezier = topo.sample('bezier', 2)
bezier2 = topo.boundary['left'].sample('bezier', 2)
x = bezier2.eval(ns.x[1])
badbnd1 = bezier2.eval(2* topo.boundary['left'].indicator(topo.boundary['left'][0:int(nelems/5)]))
print(badbnd1, x)
plt.plot(x, badbnd1, '*')
''' 

sqr = topo.boundary['bottom'].integral('(u - 1)^2 d:x' @ ns, degree = degree*2)
sqr += topo.boundary['left'].integral( '(u - badbnd)^2 d:x' @ ns, degree = degree*2 )
cons = ntl.solver.optimize('dofs', sqr, droptol=1e-15)

#residual
res = topo.integral( 'kappa basis_i,j u_,j d:x' @ ns, degree = degree * 2)
res += topo.integral( 'tau beta_k basis_i,k (-kappa u_,jj + beta_n u_,n)  d:x' @ ns, degree = degree * 2) 

lhs = ntl.solver.solve_linear('dofs', res, constrain= cons)

bezier = topo.sample('bezier', 2)
x = bezier.eval('x_i' @ ns)
u = bezier.eval('u' @ ns, dofs = lhs)


ntl.export.triplot('ADSM_SUPG_messtype.png', x, u, tri=bezier.tri, hull=bezier.hull)



#'''

optimize > solve > solving 7 dof system to machine precision using direct solver
optimize > solve > solver returned with residual 1e-16
optimize > constrained 7/16 dofs
optimize > optimum value 4.79e-02
solve > solving 9 dof system to machine precision using direct solver
solve > solver returned with residual 4e-17
ADSM_SUPG_messtype.png


In [58]:
#print((beta *beta).sum(0)) #ct.getfullargspec(topo.basis))
#elp(np.dot)#(topo.basis)
help(ntl.export.triplot)
#print(topo.boundary)
#help(bezier.eval)
a = [1 ,2, 3, 4, 5, 8, 6, 7]
print(a[:])

Help on function triplot in module nutils.export:

triplot(name, points, values=None, *, tri=None, hull=None, cmap=None, clim=None, linewidth=0.1, linecolor='k')

[1, 2, 3, 4, 5, 8, 6, 7]
